* 체류인구 데이터 정리

In [ ]:
import pandas as pd
import numpy as np
import json

import folium
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
stay1 = pd.read_csv("/content/stay1.csv")
stay2 = pd.read_csv("/content/stay2.csv")
stay3 = pd.read_csv("/content/stay_3.csv")
stay4 = pd.read_csv("/content/stay_4.csv")

In [ ]:
stay1.head()

,Unnamed: 0,hdong_cd,purpose,stay_cnts
0,0,1111051500,0,1818545
1,1,1111051500,1,541663
2,2,1111051500,2,193115
3,3,1111051500,3,505990
4,4,1111051500,4,389268


In [ ]:
combined_stay = pd.concat([stay1, stay2, stay3, stay4], ignore_index=True)
result_df = combined_stay.groupby(['hdong_cd', 'purpose'], as_index=False)['stay_cnts'].sum()

In [ ]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20798 entries, 0 to 20797
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   hdong_cd   20798 non-null  int64
 1   purpose    20798 non-null  int64
 2   stay_cnts  20798 non-null  int64
dtypes: int64(3)
memory usage: 487.6 KB


In [ ]:
kik = pd.read_csv("/content/KIKmix_20230701.csv")
kik.head()

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN


In [ ]:
kik = kik[kik['시도명'] == '인천광역시']
kik.head()

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
1652,2800000000,인천광역시,NaN,NaN,2800000000,인천광역시,19950101,NaN
1653,2811000000,인천광역시,중구,NaN,2811000000,중구,19950101,NaN
1654,2811052000,인천광역시,중구,연안동,2811011800,항동7가,19950101,NaN
1655,2811052000,인천광역시,중구,연안동,2811013800,북성동1가,19950101,NaN
1656,2811053000,인천광역시,중구,신포동,2811010100,중앙동1가,19981010,NaN


In [ ]:
kik.rename(columns = {'행정동코드' : 'hdong_cd'}, inplace = True)

In [ ]:
merged_df = pd.merge(result_df, kik[['hdong_cd', '시도명', '시군구명', '읍면동명']], on='hdong_cd', how='right')

In [ ]:
merged_df.head()

,hdong_cd,purpose,stay_cnts,시도명,시군구명,읍면동명
0,2800000000,NaN,NaN,인천광역시,NaN,NaN
1,2811000000,NaN,NaN,인천광역시,중구,NaN
2,2811052000,0.0,2878045.0,인천광역시,중구,연안동
3,2811052000,1.0,1944008.0,인천광역시,중구,연안동
4,2811052000,2.0,1926.0,인천광역시,중구,연안동


In [ ]:
b = merged_df.groupby(['hdong_cd', '시도명', '시군구명', '읍면동명', 'purpose'], as_index=False)['stay_cnts'].sum()
b

,hdong_cd,시도명,시군구명,읍면동명,purpose,stay_cnts
0,2811052000,인천광역시,중구,연안동,0.0,5756090.0
1,2811052000,인천광역시,중구,연안동,1.0,3888016.0
2,2811052000,인천광역시,중구,연안동,2.0,3852.0
3,2811052000,인천광역시,중구,연안동,3.0,413616.0
4,2811052000,인천광역시,중구,연안동,4.0,3405096.0
...,...,...,...,...,...,...
915,2872038000,인천광역시,옹진군,연평면,0.0,1913536.0
916,2872038000,인천광역시,옹진군,연평면,1.0,83866.0
917,2872038000,인천광역시,옹진군,연평면,3.0,204.0
918,2872038000,인천광역시,옹진군,연평면,4.0,469360.0


In [ ]:
#def filter_purpose(b):

#    filtered_b = b[b['purpose'].isin([3, 5])]
#    return filtered_b

#b = filter_purpose(b)
#b.head()

,hdong_cd,시도명,시군구명,읍면동명,purpose,stay_cnts
3,2811052000,인천광역시,중구,연안동,3.0,413616.0
5,2811052000,인천광역시,중구,연안동,5.0,1688420.0
9,2811053000,인천광역시,중구,신포동,3.0,20482050.0
11,2811053000,인천광역시,중구,신포동,5.0,19047900.0
15,2811054000,인천광역시,중구,신흥동,3.0,668148.0


In [ ]:
b = b.groupby(['hdong_cd', '시도명', '시군구명', '읍면동명'], as_index=False)['stay_cnts'].sum()
b

,hdong_cd,시도명,시군구명,읍면동명,stay_cnts
0,2811052000,인천광역시,중구,연안동,15155090.0
1,2811053000,인천광역시,중구,신포동,197528125.0
2,2811054000,인천광역시,중구,신흥동,57085904.0
3,2811056000,인천광역시,중구,도원동,4001453.0
4,2811057000,인천광역시,중구,율목동,2844170.0
...,...,...,...,...,...
150,2872034000,인천광역시,옹진군,대청면,3167742.0
151,2872035000,인천광역시,옹진군,덕적면,13494141.0
152,2872036000,인천광역시,옹진군,영흥면,32259224.0
153,2872037000,인천광역시,옹진군,자월면,4611620.0


* 행정동 관련 정보 추가

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

def check_if_in_incheon(csv_df, seoul_incheon_gyungi_df):
    results = []

    for idx, row in csv_df.iterrows():
        point = Point(row['lon'], row['lat'])
        found = False
        for _, incheon_row in seoul_incheon_gyungi_df.iterrows():
            if incheon_row['geometry'].contains(point):
                results.append(incheon_row['adm_nm'])
                found = True
                break

        if not found:
            results.append(None)

    csv_df['행정동'] = results
    return csv_df

csv_file_path = '4.5km이내_시설위치_관광지포함.csv'
csv_df = pd.read_csv(csv_file_path)

geojson_file_path = 'HangJeongDong_ver20241001.geojson'
geojson_df = gpd.read_file(geojson_file_path)

seoul_incheon_gyungi_df = geojson_df[geojson_df['sidonm'].isin(['인천광역시','서울특별시','경기도'])]

updated_csv_df = check_if_in_incheon(csv_df, seoul_incheon_gyungi_df)

print(updated_csv_df.head())


updated_csv_df.to_csv('행정동_위경도.csv', index=False)

   Unnamed: 0         lon        lat      장소명     구분  \
0         326  126.707067  37.383200  운전면허시험장  버스정류장   
1         327  126.706750  37.383883    면허시험장  버스정류장   
2         328  126.699233  37.393700    (주)창원  버스정류장   
3         329  126.698967  37.394117    (주)창원  버스정류장   
4         332  126.698967  37.399983    한국마쓰이  버스정류장   

                                       geometry   distance_m  distance_km  \
0   POINT (14104966.14495335 4492655.377129176)  3910.407783     3.910408   
1  POINT (14104930.890070617 4492751.105364169)  3893.048815     3.893049   
2   POINT (14104094.13485417 4494126.491242484)  4265.410780     4.265411   
3  POINT (14104064.457077924 4494184.877707844)  4288.069833     4.288070   
4  POINT (14104064.457077924 4495006.918494051)  4276.747492     4.276747   

               행정동  
0   인천광역시 연수구 송도3동  
1  인천광역시 남동구 논현고잔동  
2  인천광역시 남동구 논현고잔동  
3  인천광역시 남동구 논현고잔동  
4  인천광역시 남동구 논현고잔동  


In [ ]:
hang = pd.read_csv("/content/행정동_위경도.csv")
hang.head()

,Unnamed: 0,lon,lat,장소명,구분,geometry,distance_m,distance_km,행정동
0,326,126.707067,37.383200,운전면허시험장,버스정류장,POINT (14104966.14495335 4492655.377129176),3910.407783,3.910408,인천광역시 연수구 송도3동
1,327,126.706750,37.383883,면허시험장,버스정류장,POINT (14104930.890070617 4492751.105364169),3893.048815,3.893049,인천광역시 남동구 논현고잔동
2,328,126.699233,37.393700,(주)창원,버스정류장,POINT (14104094.13485417 4494126.491242484),4265.410780,4.265411,인천광역시 남동구 논현고잔동
3,329,126.698967,37.394117,(주)창원,버스정류장,POINT (14104064.457077924 4494184.877707844),4288.069833,4.288070,인천광역시 남동구 논현고잔동
4,332,126.698967,37.399983,한국마쓰이,버스정류장,POINT (14104064.457077924 4495006.918494051),4276.747492,4.276747,인천광역시 남동구 논현고잔동


In [ ]:
hang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   479 non-null    int64  
 1   lon          479 non-null    float64
 2   lat          479 non-null    float64
 3   장소명          479 non-null    object 
 4   구분           479 non-null    object 
 5   geometry     479 non-null    object 
 6   distance_m   479 non-null    float64
 7   distance_km  479 non-null    float64
 8   행정동          479 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 33.8+ KB


In [ ]:
b['행정동'] = b['시도명'] +' '+ b['시군구명'] +' '+ b['읍면동명']
b.head()

,hdong_cd,시도명,시군구명,읍면동명,stay_cnts,행정동
0,2811052000,인천광역시,중구,연안동,15155090.0,인천광역시 중구 연안동
1,2811053000,인천광역시,중구,신포동,197528125.0,인천광역시 중구 신포동
2,2811054000,인천광역시,중구,신흥동,57085904.0,인천광역시 중구 신흥동
3,2811056000,인천광역시,중구,도원동,4001453.0,인천광역시 중구 도원동
4,2811057000,인천광역시,중구,율목동,2844170.0,인천광역시 중구 율목동


In [ ]:
b.to_csv("목적수정한 데이터.csv")

In [ ]:
def merge_hang_a(hang, a):
    merged_df = hang.merge(b[['행정동', 'stay_cnts', 'hdong_cd']], on='행정동', how='left')
    return merged_df

merged_df = merge_hang_a(hang, b)
merged_df = merged_df[merged_df['hdong_cd'].notnull()]

In [ ]:
merged_df

,Unnamed: 0,lon,lat,장소명,구분,geometry,distance_m,distance_km,행정동,stay_cnts,hdong_cd
0,326,126.707067,37.383200,운전면허시험장,버스정류장,POINT (14104966.14495335 4492655.377129176),3910.407783,3.910408,인천광역시 연수구 송도3동,25412177.0,2.818584e+09
1,327,126.706750,37.383883,면허시험장,버스정류장,POINT (14104930.890070617 4492751.105364169),3893.048815,3.893049,인천광역시 남동구 논현고잔동,81350514.0,2.820071e+09
2,328,126.699233,37.393700,(주)창원,버스정류장,POINT (14104094.13485417 4494126.491242484),4265.410780,4.265411,인천광역시 남동구 논현고잔동,81350514.0,2.820071e+09
3,329,126.698967,37.394117,(주)창원,버스정류장,POINT (14104064.457077924 4494184.877707844),4288.069833,4.288070,인천광역시 남동구 논현고잔동,81350514.0,2.820071e+09
4,332,126.698967,37.399983,한국마쓰이,버스정류장,POINT (14104064.457077924 4495006.918494051),4276.747492,4.276747,인천광역시 남동구 논현고잔동,81350514.0,2.820071e+09
...,...,...,...,...,...,...,...,...,...,...,...
474,10749,126.748921,37.424836,플러스약국,시설,POINT (14109625.399976604 4498490.114308512),4049.154740,4.049155,인천광역시 남동구 서창2동,15031447.0,2.820066e+09
475,10750,126.735755,37.405118,하나약국,시설,POINT (14108159.745296923 4495726.491702711),1084.536304,1.084536,인천광역시 남동구 논현1동,20713247.0,2.820069e+09
476,10767,126.710347,37.402377,호구포온누리약국,시설,POINT (14105331.339674845 4495342.322219685),3073.223862,3.073224,인천광역시 남동구 논현2동,31563502.0,2.820070e+09
477,10769,126.724979,37.398741,휴먼시아약국,시설,POINT (14106960.121936338 4494832.816918367),1378.104617,1.378105,인천광역시 남동구 논현고잔동,81350514.0,2.820071e+09


In [ ]:
merged_df.to_csv("인천광역시_체류인구포함_목적_전부.csv")